# Calibration of motor current

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

from tools.data_loader import moving_average

import glob
import os

In [ ]:
# get the .csv files for the given velocity limit
def get_files_for_limit(limit, flight, discard=True):
    
    v_induced = 14.345219306215128
    v_limit = limit * 0.01 * v_induced
    data_directory = flight + f'hover_{limit}_limit'
    csv_files = glob.glob(os.path.join(data_directory, "*.csv"))
    #print(csv_files)
    sorted_files = []
    #return csv_files
    # sort sequences in ascending order
    for i in range(len(csv_files)):
        for e in csv_files:
            if f'sequence_{i}_' in e:
                #print(e)
                sorted_files.append(e)
                break
    
    # discard first and land sequence (vehicle on ground before takeoff and after landing)
    if discard:
        return sorted_files[1:-1]
    
    else:
        return sorted_files
    

In [ ]:
def get_motor_data()

In [ ]:
motor = 8

filepath_voltage = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/voltage_motor_{motor}/'
filepath_current = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/current_motor_{motor}/'

files_current = get_files_for_limit(10, filepath_current, discard=False)
files_voltage = get_files_for_limit(10, filepath_voltage, discard=False)

current_data = pd.DataFrame()
voltage_data = pd.DataFrame()

# stack sequences
for file in files_current:
    current_data = pd.concat([current_data, pd.read_csv(file)])
    
for file in files_voltage:
    voltage_data = pd.concat([voltage_data, pd.read_csv(file)])

In [ ]:
filepath_voltage_measurements = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/voltage_data.csv'
voltage_measurements = pd.read_csv(filepath_voltage_measurements)

In [ ]:
voltage_measured = voltage_measurements.iloc[motor-1].values
voltage_measured

In [ ]:
#flight_data.rename(columns = {' omega1':'omega1'}, inplace = True)

t_current = ((current_data['t'].values - min(current_data['t'].values)) / 10e5)
t_voltage = ((voltage_data['t'].values - min(voltage_data['t'].values)) / 10e5)

omega_voltage = voltage_data[f'omega{motor}'].values
omega_current = current_data[f'omega{motor}'].values

voltage = voltage_data[f'U1{motor}'].values
current = current_data[f'I1{motor}'].values

#control_voltage = voltage_data[f'ctrl_{motor}'].values
#control_current = current_data[f'ctrl_{motor}'].values

In [ ]:
# get count of unique values
unique_rpms_voltage = np.unique(omega_voltage[omega_voltage>=1000], return_counts=True) # all unique RPMs above 1000
filtered_rpms_voltage = unique_rpms_voltage[0][unique_rpms_voltage[1] > 20] # Only RPM that occur more than 20 times
filtered_voltage_indices = np.nonzero(np.isin(omega_voltage, filtered_rpms_voltage)==1)[0] # apply filter

# get count of unique values
unique_rpms_current = np.unique(omega_current[omega_current>=1000], return_counts=True) 
filtered_rpms_current = unique_rpms_current[0][unique_rpms_current[1] > 20]
filtered_current_indices = np.nonzero(np.isin(omega_current, filtered_rpms_current)==1)[0]

# get start and ends of sequence

In [ ]:
mask = np.where(np.diff(filtered_voltage_indices,prepend=0)>10) # search for the gaps between the sequences
start_voltage = filtered_voltage_indices[mask]

mask = np.where(np.abs(np.diff(np.flip(filtered_voltage_indices),prepend=0))>10)
end_voltage = np.flip(np.flip(filtered_voltage_indices)[mask])

mask = np.where(np.diff(filtered_current_indices,prepend=0)>10) # search for the gaps between the sequences
start_current = filtered_current_indices[mask]

mask = np.where(np.abs(np.diff(np.flip(filtered_current_indices),prepend=0))>10)
end_current = np.flip(np.flip(filtered_current_indices)[mask])

In [ ]:
# get index arrays for every sequence
volt_sequences = start_voltage.size
curr_sequences = start_current.size
seq_idx_current = []
seq_idx_voltage = []

for i in range(volt_sequences):
    
    intersection = np.intersect1d(np.arange(start_voltage[i],end_voltage[i]), filtered_voltage_indices)
    
    if not intersection.size == 0:
        seq_idx_voltage.append(intersection[5:-5]) # remove first and last 5 values to get rid of outliers

for i in range(curr_sequences):
    
    intersection = np.intersect1d(np.arange(start_current[i],end_current[i]), filtered_current_indices)
    
    if not intersection.size == 0:
        seq_idx_current.append(intersection[5:-5]) # remove first and last 5 values to get rid of outliers

In [ ]:
# motor cmds

fig0, ax0 = plt.subplots(2,2, figsize=(10,8))

# Rotor pairs

# raw data
ax0[0][0].scatter(t_voltage, omega_voltage, color='lightgray', label='RPM raw')
ax0[0][1].scatter(t_voltage, voltage, color='lightgray', label='esc-voltage raw')

ax0[1][0].scatter(t_current[filtered_current_indices], omega_current[filtered_current_indices])

#ax0[0][0].scatter(t_voltage[filtered_voltage_indices], omega_voltage[filtered_voltage_indices], label='RPM')


ax0[1][1].scatter(t_current[filtered_current_indices], current[filtered_current_indices], label='current')

# plot the sequences 

for i, seq in enumerate(seq_idx_voltage):
    
    
    t = (max(t_voltage[seq]) + min(t_voltage[seq]))/2
    mean_voltage = np.mean(voltage[seq])
    
    if(i==0):
        ax0[0][0].scatter(t_voltage[seq], omega_voltage[seq], color='deepskyblue', label = 'RPM filtered')
        ax0[0][1].scatter(t_voltage[seq], voltage[seq], color='deepskyblue', label='esc-voltage filtered')
        ax0[0][1].scatter(t, mean_voltage, s=50, marker='x', color='b', label='esc-voltage mean')
        ax0[0][1].scatter(t, voltage_measured[i], s=50, marker='x', color='r',label='multimeter-voltage')
    else:
        ax0[0][0].scatter(t_voltage[seq], omega_voltage[seq], color='deepskyblue')
        ax0[0][1].scatter(t_voltage[seq], voltage[seq], color='deepskyblue')
        ax0[0][1].scatter(t, mean_voltage, s=50, marker='x', color='b')
        ax0[0][1].scatter(t, voltage_measured[i], s=50, marker='x', color='r')

for i in range(2):
    for j in range(2):
        ax0[i][j].set_xlabel('Time [s]')
        #ax0[i][j].set_xlim(0,130)
        ax0[i][j].grid()
        ax0[i][j].legend()

ax0[0][0].set_ylabel('$RPM$')
ax0[0][1].set_ylabel('Voltage')

ax0[0][0].set_ylim(0,4500)
ax0[1][0].set_ylim(0,4500)

ax0[0][0].set_title(f'RPM over time')
ax0[0][1].set_title(f'Voltage over time')
ax0[1][0].set_title(f'RPM over time')
ax0[1][1].set_title(f'Current over time')
fig0.tight_layout()